## ĐỒ ÁN CUỐI KÌ
****

### Thành viên
- 19127642&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Võ Nhất Huy
- 19127592&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Lê Minh Trí
### Dataset
- HR Analytics: Job Change of Data Scientists
- Link: https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists
***

In [1]:
import pandas as pd
import numpy as np

In [2]:
aug_df = pd.read_csv('aug_train.csv')

In [3]:
aug_df

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [5]:
n_rows = len(aug_df)
n_columns = len(aug_df.columns)
print(n_rows, n_columns)

19158 14


In [6]:
aug_df.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

In [7]:
print(aug_df.duplicated().any())

False


In [15]:
column_name = aug_df.columns
col_missing_count = []
for col in column_name:
    col_missing_count.append(aug_df[col].isna().sum())
missing_dict = dict(zip(column_name, col_missing_count))
for key, value in missing_dict.items():
    print(key, ':', value, '|', value/n_rows*100)

enrollee_id : 0 | 0.0
city : 0 | 0.0
city_development_index : 0 | 0.0
gender : 4508 | 23.530639941538784
relevent_experience : 0 | 0.0
enrolled_university : 386 | 2.0148240943731075
education_level : 460 | 2.401085708320284
major_discipline : 2813 | 14.68316108153252
experience : 65 | 0.33928385008873574
company_size : 5938 | 30.994884643490973
company_type : 6140 | 32.04927445453596
last_new_job : 423 | 2.207954901346696
training_hours : 0 | 0.0
target : 0 | 0.0


### Tiền xử lí dữ liệu

### Đặt câu hỏi

### Trả lời câu hỏi